In [1]:
%matplotlib inline

In [ ]:
# A lot of inspiration from https://github.com/loudinthecloud/pytorch-ntm. Hyperparameters were chosen based
# upon his experiments.

import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import torch.utils.data as d
import random

In [45]:
def _split_cols(mat, lengths):
    """Split a 2D matrix to variable length columns."""
    assert mat.size()[1] == sum(lengths), "Lengths must be summed to num columns"
    l = np.cumsum([0] + lengths)
    results = []
    for s, e in zip(l[:-1], l[1:]):
        results += [mat[:, s:e]]
    return results

class NTM_Head(nn.Module):
    def __init__(self, memory, controller_output_size):
        super(NTM_Head, self).__init__()
        
        self.memory = memory
        self.controller_output_size = controller_output_size
        self.N, self.M = memory.size()
    
    def is_read_head(self):
        raise NotImplementedError
    
    def initialize_parameters(self):
        raise NotImplementedError
    
    def initialize_state(self):
        raise NotImplementedError
        
    def address_memory(self, key_vec, β, g, s, γ):
        result = F.cosine_similarity(key_vec.unsqueeze(0).expand(self.N, -1), self.memory, dim = 0)
        result = β * result
        result = result.exp() / result.sum()
        result = g * result + (1 - g) * self.prev_address_vector
        result = F.conv1d(torch.cat((result[:, -1], result, result[:, 1]), 1).view(-1, 1, self.N), 
                          s.view(1, 1, -1))
        result = result ** γ
        result = result / result.sum()
        return result        

In [ ]:
class NTM_Read_Head(NTM_Head):
    def __init__(self, memory, controller_output_size):
        super(NTM_Read_Head).__init__(memory, controller_output_size)
        
        self.read_parameters_lengths = [self.N, 1, 1, 3, 1]
        self.fc_read_parameters = nn.Linear(controller_output_size, sum(read_parameters_lengths))
        
        initialize_parameters()
        initialize_state()
    
    def reset_parameters(self):
        nn.init.xavier_uniform(self.fc_read_parameters.weight, gain=1.4)
        nn.init.normal(self.fc_read_parameters.bias, std=0.01)
    
    def initialize_state(self):
        self.initial_address_vector = nn.Parameter(torch.zeros(self.N))
        self.initial_read = nn.Parameter(torch.zeros(self.M))
        
        self.prev_address_vector = Variable(self.initial_address_vector)
        self.prev_read = Variable(self.initial_read)
    
    def is_read_head(self):
        return True
    
    def forward(self, x):
        read_parameters = self.fc_read_parameters(x)
        
        key_vec, β, g, s, γ = _split_cols(read_parameters, self.read_parameters_length)
        β = F.softplus(β)
        g = F.sigmoid(g)
        s = F.softmax(s)
        γ = 1 + F.softplus(γ)
                                               
        address_vec = address_memory(key_vec, β, g, s, γ)
        new_read = self.M.transpose() * address_vec
        self.prev_read = new_read
        return new_read

In [ ]:
class NTM_Write_Head(NTM_Head):
    def __init__(self, memory, controller_output_size):
        super(NTM_Write_Head).__init__(memory, controller_output_size)
        
        self.write_parameters_lengths = [self.N, 1, 1, 3, 1, self.M, self.M]
        self.fc_write_parameters = nn.Linear(controller_output_size, sum(write_parameters_size))
        
        reset_parameters()
        initialize_state()
    
    def reset_parameters(self):
        nn.init.xavier_uniform(self.fc_write_parameters.weight, gain=1.4)
        nn.init.normal(self.fc_write_parameters.bias, std=0.01)
    
    def initialize_state(self):
        self.initial_address_vector = nn.Parameter(torch.zeros(self.N))       
        self.prev_address_vector = Variable(self.initial_address_vector)
    
    def is_read_head(self):
        return False
    
    def forward(self, x):
        write_parameters = self.fc_write_parameters(x)
                                       
        key_vec, β, g, s, γ, erase_vec, add_vec = _split_cols(write_parameters, self.write_parameters_size)
        β = F.softplus(β)
        g = F.sigmoid(g)
        s = F.softmax(s)
        γ = 1 + F.softplus(γ)
        erase_vec = F.sigmoid(erase_vec)
                                               
        address_vec = address_memory(key_vec, β, g, s, γ)
        self.M *= 1 - torch.bmm(address_vec.unsqueeze(2), erase_vec.unsquueze(1))
        self.M += torch.bmm(address_vec.unsqueeze(2), add_vec.unsquueze(1))

In [ ]:
class NTM(nn.Module):
    def __init__(self, controller, controller_output_size, output_size, 
                 address_count, address_dimension, heads):
        super(NTM, self).__init__()
        
        # Initialize memory
        self.memory = nn.Parameter(torch.zeros(address_count, address_dimension))
        
        # Construct the heads.
        self.heads = nn.ModuleList()
        
        for head_id in heads:
            if head_id = 0:
                self.heads.append(NTM_Read_Head(memory, controller_output_size))
            else:
                self.heads.append(NTM_Write_Head(memory, controller_output_size))
        
        # Initialize controller
        self.controller = controller
        self.outputGate = nn.Linear(controller_output_size, output_size)
        self.softmax = nn.Softmax()
        
        initialize_state()
        reset_parameters()
    
    def initialize_state(self):
        self.prev_reads = []
        
        for head in self.heads:
            if head.is_read_head():
                self.prev_reads.append(head.prev_read)
        
    def reset_parameters(self):
        stdev = 1 / (np.sqrt(N + M))
        nn.init.uniform(self.memory, -stdev, stdev)
        
    def forward(self, x):
        controller_output = controller(torch.cat(self.prev_reads.append(x), dim=1))
        self.prev_reads = []
        
        for head in self.heads:
            if head.is_read_head():
                self.prev_reads.append(head(controller_output))
            else:
                head(controller_output)
        
        return self.softmax(self.outputGate(controller_output))

In [ ]:
class CopyTaskDataset(d.Dataset):
    def __init__(self, batch_size, lower, upper, seq_size):
        self.input_list = []
        for _ in range(batch_size):
            self.input_list.append(self.generate_batch(batch_size, lower, upper, seq_size))
        
        self.batch_size = batch_size

    def generate_batch(self, batch_size, lower, upper, seq_size):
        seq_length = random.randint(lower, upper)
        batch = torch.from_numpy(
            numpy.random.binomial(1, 0.5, (seq_length, batch_size, seq_size)))
        end_marker = torch.zeros(seq_length - 1, batch_size, 1)
        end_row = torch.ones(1, batch_size, 1)
        end_marker = torch.cat((end_marker, end_row), 0)
        batch = torch.cat((batch.float(), end_marker), 2)
        print(batch)
        return batch 
    
    def __len__(self):
        return len(self.input_list)
    
    def __getitem__(self, i):
        return self.input_list[i//self.batch_size][:, i % self.batch_size, :]

In [ ]:
def trainAll(num_batches, batch_size, lower_seq_length=3, 
             upper_seq_length=10, seq_size=8):
    ntm = NTM(1,1,1,1,1)   
    dataset = CopyTaskDataset(batch_size, lower_seq_length, upper_seq_length, seq_size)
    
    data_loader = d.DataLoader(dataset, batch_size = batch_size)
    for _ in range(num_batches):
        for batch in data_loader:
            batch = batch.squeeze()
            # Pass in one element of the sequence per time step
            for time_step in range(len(batch)):
                #ntm(batch[time_step], None)
                pass
                
            # Now, don't pass in any elements.
            for time_step in range(len(batch)):
                ntm(None, batch[time_step][:,:-1])
    print("DONE!")

In [ ]:
class EncapsulatedLSTM(nn.Module):
    def __init__(self, batch_size, *args, **kwargs):
        super(EncapsulatedLSTM, self).__init__()
        self.lstm = nn.LSTM(*args, **kwargs)
        hidden_size = args[1]
        num_layers = args[2]
        
        initial_hidden_state = nn.Parameter(torch.randn(num_layers, 1, hidden_size))
        initial_hidden_state.expand(num_layers, batch_size, hidden_size)
        initial_cell_state = nn.Parameter(torch.randn(num_layers, 1, hidden_size))
        initial_cell_state.expand(num_layers, batch_size, hidden_size)
        self.state_tuple = (initial_hidden_state, initial_cell_state)
        
    def forward(self, input):
        output, self.state_tuple = self.lstm(input, self.state_tuple)
        return output